## BERT Set

Initial work found in [this BERT Sentence Embedding tutorial](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#sentence-vectors)

In [2]:
import pickle

import nltk
import torch
from transformers import BertTokenizer, BertModel

def marked_text(input_sent):
    return "[CLS]" + input_sent + "[SEP]"

#token_vecs = hidden_states[-2][0]
#sentence_embedding = torch.mean(token_vecs, dim=0)

In [4]:
with open(f"../data/texts_labelled_sub100.pickle", mode='rb') as filein:
    texts = pickle.load(filein)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

In [ ]:
model.eval();

First, be sure to initialize `BertModel` with `output_hidden_states=True`

For each article:
* We call `nltk.sent_tokenize` on its scraped text,
* then we sandwich each sentence above between the `[CLS]` and `[SEP]` tags
* then we call `BertTokenizer.tokenize` on each sentence of the above,
* then we call `BertTokenizer.convert_tokens_to_ids` on each tokenized text above forming `indexed_tokens`,
* then we assign a segment ID of `1` to all of the tokens in each individual sentence, forming `segment_ids`
    * since we will be passing each sentence through without the objective of comparing them (they all come from the same document)
* then we convert both `indexed_tokens` and `segment_ids` into `torch.tensor`s
    * `x = torch.tensor([x])`
* then, for each pair of tokenized text and segment ID array, we run it through a forward pass in the pretrained `BertModel` making sure to initialize it with `output_hidden_states=True`
* 

In [ ]:
#tokenized_text = tokenizer.tokenize(marked_text)
#indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#
#tokens_tensor = torch.tensor([indexed_tokens])
#segments_ids = [1] * len(tokenized_text)
#segments_tensors = torch.tensor([segments_ids])

In [ ]:
#with torch.no_grad():
#
#    outputs = model(tokens_tensor, segments_tensors)
#
#    # Evaluating the model will return a different number of objects based on 
#    # how it's  configured in the `from_pretrained` call earlier. In this case, 
#    # because we set `output_hidden_states = True`, the third item will be the 
#    # hidden states from all layers. See the documentation for more details:
#    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
#    hidden_states = outputs[2]